In [31]:
import mlflow
from mlflow.tracking import MlflowClient
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import joblib

In [32]:
#server_uri = "http://localhost:5000"
server_uri = "http://antoniomlflowaci.westeurope.azurecontainer.io:5000"
exp_name = "tree_model_iris"
model_name = "tree_model"
artifact_path = "mlflow-artifact-root"
params = {"max_depth": 3, "max_features": 2, "min_samples_split": 20, "random_state": 1}

In [33]:
mlflow.set_tracking_uri(server_uri)
client = MlflowClient()
exp_names = [exp.name for exp in client.list_experiments()]
if exp_name not in exp_names:
    client.create_experiment(exp_name)
exp = client.get_experiment_by_name(exp_name)

In [34]:
iris = datasets.load_iris()
x, y = iris.data, iris.target
labels = iris.target_names
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=1)

In [35]:
# If issues with azure dependencies:
# pip uninstall -y azure-common azure-storage azure-nspkg azure-storage-blob
# pip install azure-storage-blob --upgrade
# pip install azure-common --upgrade

In [ ]:
#%env AZURE_STORAGE_ACCESS_KEY=""

In [37]:
with mlflow.start_run(experiment_id=exp.experiment_id) as run:
    tree = DecisionTreeClassifier(**params)
    tree.fit(x_train, y_train)
    y_pred = tree.predict(x_test)

    mlflow.log_params(tree.get_params())
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))
    mlflow.sklearn.log_model(tree, artifact_path)
    model_uri = f"runs:/{run.info.run_id}/{artifact_path}"
    mv = mlflow.register_model(model_uri, "tree_model")

Successfully registered model 'tree_model'.
2021/09/06 13:30:28 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: tree_model, version 1
Created version '1' of model 'tree_model'.


In [38]:
mlflow.sklearn.load_model("models:/tree_model/1")

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=3, max_features=2, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=20,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1, splitter='best')

In [26]:
import requests
import json

In [ ]:
feat_names = iris.feature_names
dummy_data = [5.8, 4. , 1.2, 0.2]
dummy_request = {feat_names[i]: dummy_data[i] for i in range(len(feat_names))}
test = {"sepal_length": 2.1,  "sepal_width": 0.2,  "petal_length": 4.3, "petal_width": 2}

#url = "http://localhost:8000/predict_iris"
url = "http://antonioirismodelaci.westeurope.azurecontainer.io:8000/predict_iris"

prediction = requests.get(url, {"flower_data": json.dumps(test)})

In [ ]:
prediction.text